In [3]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import colorcet as cc
import torchvision

from utils import load_image
fr

In [5]:
model = torchvision.models.inception_v3(pretrained=True)
image = load_image('images/doberman.png', size=299)

In [6]:
guided_backprop = GuidedBackprop(model)
gb_mask = guided_backprop.get_smoothed_mask(image.unsqueeze(0), target_class)
show_mask(gb_mask, cmap=cc.cm.gray)
guided_backprop.remove_hooks()

NameError: name 'GuidedBackprop' is not defined